In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Thompson,CA,2020-01-14 04:47:21,55.74,-97.86,-7.60,76,75,6.93
1,1,Hilo,US,2020-01-14 04:47:21,19.73,-155.09,75.99,86,78,11.01
2,2,Shingu,JP,2020-01-14 04:47:22,33.73,135.98,55.40,43,40,5.82
3,3,Vila Velha,BR,2020-01-14 04:46:50,-20.33,-40.29,78.80,88,75,11.41
4,4,Avarua,CK,2020-01-14 04:47:22,-21.21,-159.78,80.60,78,76,13.87


In [5]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [16]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [30]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [32]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Hilo,US,2020-01-14 04:47:21,19.73,-155.09,75.99,86,78,11.01
3,3,Vila Velha,BR,2020-01-14 04:46:50,-20.33,-40.29,78.80,88,75,11.41
4,4,Avarua,CK,2020-01-14 04:47:22,-21.21,-159.78,80.60,78,76,13.87
7,7,Carnarvon,AU,2020-01-14 04:37:23,-24.87,113.63,79.84,77,91,17.58
9,9,Sao Jose Da Coroa Grande,BR,2020-01-14 04:38:15,-8.90,-35.15,80.46,71,28,8.55
17,17,Sao Joao Da Barra,BR,2020-01-14 04:45:58,-21.64,-41.05,75.11,90,100,9.64
18,18,Rikitea,PF,2020-01-14 04:47:24,-23.12,-134.97,78.17,82,2,10.83
22,22,Bandarbeyla,SO,2020-01-14 04:38:20,9.49,50.81,76.10,69,22,21.18
28,28,New Norfolk,AU,2020-01-14 04:47:27,-42.78,147.06,80.01,44,20,11.41
33,33,Mount Gambier,AU,2020-01-14 04:47:28,-37.83,140.77,82.40,48,8,13.87


In [25]:
preferred_cities_df.count()

City_ID       147
City          147
Country       147
Date          147
Lat           147
Lng           147
Max Temp      147
Humidity      147
Cloudiness    147
Wind Speed    147
dtype: int64

In [38]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hilo,US,75.99,19.73,-155.09,
3,Vila Velha,BR,78.80,-20.33,-40.29,
4,Avarua,CK,80.60,-21.21,-159.78,
7,Carnarvon,AU,79.84,-24.87,113.63,
9,Sao Jose Da Coroa Grande,BR,80.46,-8.90,-35.15,
...,...,...,...,...,...,...
550,Ewa Beach,US,78.80,21.32,-158.01,
552,Batemans Bay,AU,77.00,-35.72,150.18,
561,Namatanai,PG,84.06,-3.67,152.43,
563,Nangomba,TZ,78.73,-10.90,38.50,


In [40]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [47]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

# Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
     print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [50]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hilo,US,75.99,19.73,-155.09,Hilo Hawaiian Hotel
3,Vila Velha,BR,78.80,-20.33,-40.29,Hotel Vitória Palace
4,Avarua,CK,80.60,-21.21,-159.78,Paradise Inn
7,Carnarvon,AU,79.84,-24.87,113.63,Hospitality Carnarvon
9,Sao Jose Da Coroa Grande,BR,80.46,-8.90,-35.15,Crôa Mares Hotel
17,Sao Joao Da Barra,BR,75.11,-21.64,-41.05,Pousada Porto De Canoas
18,Rikitea,PF,78.17,-23.12,-134.97,Pension Maro'i
22,Bandarbeyla,SO,76.10,9.49,50.81,REER XOOSHLE GROUP
28,New Norfolk,AU,80.01,-42.78,147.06,The Shingles Riverside Cottages
33,Mount Gambier,AU,82.40,-37.83,140.77,Best Western Southgate Motel


In [54]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [60]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [61]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))